<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [14]</a>'.</span>

#Additive Manufacturing Path Planning Made Effortless

--> ***(This notebook is built up on the existing nimplex software and examples. Please see https://github.com/amkrajewski/nimplex for setting up and utilizing nimplex)***

--> You will additionally need `pycalphad` and `pathfinding` libraries to run part of this exercise. If you are running this in Codespaces, it has been pre-installed for you.

**In this tutorial, we will demonstrate how effortless it is to dramatically speed up the exploration of feasible compositional spaces in high dimensional spaces through employing `nimplex`'s graph representations that abstract the underlying problem and dimensionality.**

**We will also design several neat, mathematically optimal (given some criteria) paths in a 7-component chemical space connecting two alloys of interest by mixing 4 fixed-composition alloy powders to create a tetrahedral attainable/design space. The beauty of this approach is that at no point (except for plotting in 3D for "human consumption") will we explicitly consider the dimensionality or the distance as the connectivity between the points in the space has been abstracted into graph adjacency. If you wish to add another alloy to the design process, you add it to the list, and you are done :)**

In [1]:
# Import nimplex and some of its plotting utilities
import nimplex
from utils import plotting

In [2]:
# Python wrapper for Plotly library
import plotly.express as px
import pandas as pd
from pprint import pprint

## Example
Let's get back to our example (QuickStart) of the pair of **attainable space** simplex grid and corresponding **elemental space** positions defined for `7`-component elemental space of `Ti`, `Zr`, `Hf`, `W`, `Nb`, `Ta`, and `Mo` formed by `4` alloys:
- Ti50 Zr50 
- Hf95 Ti5
- Mo33 Nb33 Ta33
- Mo10 Nb10 W80

which we can represent as points in the elemental space:

In [3]:
elementalSpaceComponents = ['V', 'Ni', 'Cr', 'Fe']
attainableSpaceComponents = ['SS304L', 'NiCr', 'V', 'FE']
attainableSpaceComponentPositions = [[0, 0.096114519430,0.1993865031, 0.7044989775], [0, 80, 20, 0], [100, 0, 0, 0], [0, 0, 0, 100]]

And create tetrahedral grids with their compositions quantized at `12` divisions per dimension.

In [4]:
gridAtt, gridEl = nimplex.embeddedpair_simplex_grid_fractional_py(attainableSpaceComponentPositions, 12)

We used the **elemental**, or chemical, space to run the Root Mean Square Atomic Displacement (RMSAD) model by Tandoc (10.1038/s41524-023-00993-x) which acts as a lower-cost proxy for yield stress and hardness estimations in the absence of direct data:

#UNMARKDOWN WHEN NEEDED
import pqam_rmsadtandoc2023
rmsadList = []
for point in gridEl:
    formula = ' '.join([f'{c}{p}' for c, p in zip(elementalSpaceComponents, point)])
    rmsadList.append(pqam_rmsadtandoc2023.predict(formula))

And the **attainable** space grid to plot the results after projecting them to the Euclidean space:

In [5]:
# Hover approximate formula for each point
formulas = []
for i, comp in enumerate(gridEl):
    formulas.append(f"({i:>3}) "+"".join([f"{el}{100*v:.1f} " if v>0 else "" for el, v in zip(elementalSpaceComponents, comp)]))

# Generate the projected grid
gridAtt_projected_df = pd.DataFrame(plotting.simplex2cartesian_py(gridAtt), columns=['x','y','z'])

# Attach pure component (alloy) labels to corners
pureComponentIndices = nimplex.pure_component_indexes_py(4, 12)
labels = ['']*len(gridAtt_projected_df)
for comp, idx in zip(attainableSpaceComponents, pureComponentIndices):
    labels[idx] = "<b>"+comp+"</b>"


# Add text labels at the corners of the simplex
px.scatter_3d(gridAtt_projected_df, x='x', y='y', z='z', color=rmsadList, text=labels, hover_name=formulas,
              template='plotly_white', width=800, height=700, 
              labels={'color':'RMSAD', 'x':'', 'y':'', 'z':''})

## Thermodynamic Equilibria

**In this section we will be using `pycalphad` which is an amazing free open-source library for building thermodynamic models and performing calculations on them, including studying of phase equilibria, or what atomic arrangements are stable (coexist in equilibrium) at a given temperature and composition.** You can read more about `pycalphad` at [pycalphad.org](https://pycalphad.org/) and find an great tutorial at [this 2023 workshop materials repository](https://github.com/materialsgenomefoundation/2023-workshop-material/tree/main/pycalphad).

Let's start by loading a database of thermodynamic properties which defines, among other things, the Gibbs energy of each phase as a function of temperature and composition. You can find many of such databases at [TDBDB](https://avdwgroup.engin.brown.edu) maintained by Axel van de Walle's group at Brown University. 

In this example, we will be using a TDB `CrHfMoNbTaTiVWZr_9element_Feb2023` placed for you in the `examples` directory, which is a 9-element database for the elements `Cr`, `Hf`, `Mo`, `Nb`, `Ta`, `Ti`, `V`, `W`, and `Zr`, being developed by Shuang Lin in our group (Phases Research Lab at PSU). This wersion is an older *work in progress* one that has not been published, so while it is perfect for tutorial like this one, please refrain from using it for any serious work.

In [6]:
from pycalphad import Database
dbf = Database("ammap/databases/Al-Cr-Fe-Ni-Ti-V_04-13.tdb")
phases = list(set(dbf.phases.keys()))
print(elementalSpaceComponents)
print(f'Loaded TDB file with phases considered: {phases}')

['V', 'Ni', 'Cr', 'Fe']
Loaded TDB file with phases considered: ['GAMMA_H', 'AL3NI2', 'AL2FE', 'ALCR2', 'HCP_A3', 'NIV3_A15', 'ALCU_ZETA', 'LAVES_A15', 'AL13FE4', 'AL45V7', 'AL11CR4', 'AL3TI_L', 'AL5FE2', 'ALCU_THETA', 'TS01T3', 'GAMMA_D83', 'LAVES_C14', 'AL4CR', 'AL5CR', 'NI3TI', 'AL45CR7_12', 'AL5TI3', 'ALTI', 'LAVES_C36', 'BCC_B2', 'L12_FCC', 'AL23V4', 'AL4NI3', 'AL3NI5', 'NI8V', 'AL5TI2', 'GAMMAH', 'AL21V2_227', 'AL8V5', 'SIGMA_D8B', 'ALTI3_D019', 'FCC_A1', 'NITI2', 'LIQUID', 'GAMMAL', 'AL3M', 'AL45CR7', 'ALCU_DELTA', 'ALCU_EPSILON', 'D82_217', 'TAOA', 'BCC_A2', 'TS01TI', 'CRNI2', 'AL5FE4', 'ALCU_ETA', 'AL23V4_194', 'AL21V2', 'LAVES_C15', 'AL2TI', 'AL3NI1', 'NI3V_D022']


/ocean/projects/dmr190011p/arichte1/micromamba/envs/ammap/lib/python3.11/site-packages/pycalphad/io/tdb.py:294: UserWarning: The type definition character `*` in `TYPE_DEFINITION * GES AMEND_PHASE_DESCRIPTION HCP_A3 MAGNETIC -3.0 2.80000E-01 ` is not used by any phase.
  warnings.warn(f"The type definition character `{typechar}` in `TYPE_DEFINITION {typechar} {line}` is not used by any phase.")


As you can see, we will be looking at several different phases here. To keep things as simple as possible, we can split them in three groups:
- **liquid** phases: `LIQUID` (which we obviously want to avoid in a solid part design)
- **solid solution** phases: `FCC_A1`, `BCC_A2`, and `HCP_A3`
- **intermetallic** phases: `LAVES_C14`, `LAVES_C15`, and `LAVES_C36`

Designing an alloy is a complex procedure but for the sake of this tutorial, we will be focusing on the most common issue which is the formation of intermetallic phases which cause embrittlement and reduce the ductility of the material. **Thus, we will apply a simple constraint that the alloy should only contain the solid solution phases FCC, BCC, and HCP.**

**Now, knowing what we are looking for (phases at equilibrium), we can start by writing a short Python script around `pycalphad` to calculate the equilibrium phases for a given chemical elements composition `elP`. It is already placed in the `examples` directory as `myPycalphadCallable.py` which defines a function `equilibrium_callable` that takes a composition and returns the equilibrium phases.**

**We will arbitrarily pick `1000K`** as the temperature for the sake of this tutorial, but you can change it to any other value you like. Or even make it a list and add phases present at each temperature to a set to apply our constraint over a range of temperatures.


Please note that much more information is generated in the process (e.g., chemical composition of each phase and its fraction) but we are only interested in the phase presence. If you wish to do so, modifying the script to, e.g., allow for up to 5% of intermetallic phases, is a trivial task. Advanced users may also want to have a look at the `scheil_callable` we do not use in this tutorial for the sake of runtime, but which can be used to simulate solidification of the alloy from a liquid state in an additive manufacturing process.

In [7]:
from ammap.callables.EqScheil import equilibrium_callable, scheil_callable

Let's test it on some composition in our space starting with the first point!

In [8]:
print(formulas[0])
equilibrium_callable(gridEl[0])

(  0) Fe100.0 


['BCC_B2']

You should see `['BCC_A2']` in a second or so if you've run it at the default `1000K`. Quick and neat, right? Now, let's pick some compositionally complex alloy that does not lay around the corner of the attainable space tetrahedron and presents an actual challenge.

In [9]:
print(formulas[63])
equilibrium_callable(gridEl[63])

( 63) Ni40.0 Cr10.0 Fe50.0 


['L12_FCC']

In [10]:
scheil63=scheil_callable(gridEl[63])
print(scheil63)

/ocean/projects/dmr190011p/arichte1/micromamba/envs/ammap/lib/python3.11/site-packages/pycalphad/core/utils.py:120: FutureWarning: generate_dof is deprecated and will be removed in a future version of pycalphad. The correct way to determine the degrees of freedom of a particular 'active' phase is to use Model.constituents.
  warnings.warn(msg, FutureWarning)


['FCC_A1']


Now, you should have seen an example of infeasible point composed of `['HCP_A3', 'LAVES_C15', 'BCC_A2']`. Let's deploy this in parallel over all the points in the elemental space `gridEl` and see how it looks like! We will use the `process_map` function from the `tqdm` library to show a neat progress bar while the calculations are running in parallel. On the 4-core Codespaces VM you can expect it to take around 2-3 minutes.

In [11]:
from tqdm import tqdm
from tqdm.contrib.concurrent import process_map

In [12]:
gridPhases = process_map(equilibrium_callable, gridEl)

  0%|          | 0/455 [00:00<?, ?it/s]

In [13]:
gridPhases[120:130]

[[],
 ['SIGMA_D8B', 'SIGMA_D8B'],
 ['SIGMA_D8B', 'SIGMA_D8B', 'BCC_B2'],
 ['BCC_B2', 'SIGMA_D8B'],
 ['L12_FCC'],
 ['L12_FCC', 'BCC_B2'],
 ['BCC_B2', 'L12_FCC'],
 ['BCC_B2', 'L12_FCC'],
 ['BCC_B2', 'L12_FCC', 'SIGMA_D8B'],
 ['BCC_B2', 'SIGMA_D8B']]

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [14]:
scheilPhases = process_map(scheil_callable2,gridEl)

NameError: name 'scheil_callable2' is not defined

In [ ]:
# Write gridPhases to JSON file
import json

# Write scheilmap to JSON file
with open('scheilmap.json', 'w') as f:
    json.dump(scheilPhases, f)

# Write gridPhases to JSON file
with open('gridPhases.json', 'w') as f:
    json.dump(gridPhases, f)

Let's see how some of the data looks like.

Now, let's turn that list of phases into a list of feasibility based on the constraint we defined earlier. Note that in some cases, the `pycalphad` library may return an empty list of phases, which we will treat as infeasible.

In [ ]:
infeasiblePhases = set([
    'AL3M', 'AL5FE2', 'LIQUID', 'AL11CR4', 'D82_217', 'AL3NI1', 'NI3TI', 'AL5FE4', 'TAOA', 'LAVES_C15', 'GAMMAH', 'AL5CR', 'AL2FE', 'AL4CR', 'NIV3_A15', 'GAMMA_H', 
    'ALTI3_D019', 'ALCU_DELTA', 'AL45CR7', 'TS01TI', 'NI8V', 'AL45CR7_12', 'LAVES_C14', 'LAVES_A15', 'AL3NI5', 'AL5TI3', 'ALCU_EPSILON', 'NITI2', 'AL8V5', 'ALCR2', 'AL23V4', 'AL13FE4', 
    'AL4NI3', 'ALCU_ZETA', 'ALCU_THETA', 'AL3NI2', 'AL23V4_194', 'CRNI2', 'NI3V_D022', 'LAVES_C36', 'TS01T3', 'AL21V2_227', 'AL5TI2', 'ALCU_ETA', 'GAMMAL', 
    'GAMMA_D83', 'SIGMA_D8B', 'AL21V2', 'AL2TI', 'AL3TI_L', 'ALTI', 'AL45V7'])
print(f"Feasible phases set: {set(phases).difference(infeasiblePhases)}")
gridFeasible = [len(set(p) & infeasiblePhases)==0 and p!=[] for p in gridPhases]
gridFeasible[120:130]
scheilFeasible= [len(set(p) & infeasiblePhases)==0 and p!=[] for p in scheilPhases]

Finally, let's plot the result in 3D using the `plotly` library and our spacial-transformed attainable space grid we obtained with `plotting.simplex2cartesian_py(gridAtt)` earlier.

**Once you run the cell below, you should be seeing an interactive 3D plot with 455 split roughly 50/50 between feasible and infeasible points. You can rotate the plot, zoom in and out, and hover over the points to see their composition and feasibility. You can also click on the legend to hide/show the points based on their feasibility.**

In [ ]:
fig = px.scatter_3d(gridAtt_projected_df, x='x', y='y', z='z', color=gridFeasible, text=labels, hover_name=formulas,
              template='plotly_white', width=800, height=700, opacity=0.333, color_discrete_sequence=['green', 'red'],
              labels={'color':'Solid Solution Phases', 'x':'', 'y':'', 'z':''})
fig.update_scenes({'camera': {'eye': {'x': -2.3, 'y': 0.2, 'z': 0.2}}})

In [ ]:
fig = px.scatter_3d(gridAtt_projected_df, x='x', y='y', z='z', color=scheilFeasible, text=labels, hover_name=formulas,
              template='plotly_white', width=800, height=700, opacity=0.333, color_discrete_sequence=['green', 'red'],
              labels={'color':'Solid Solution Phases', 'x':'', 'y':'', 'z':''})
fig.update_scenes({'camera': {'eye': {'x': -2.3, 'y': 0.2, 'z': 0.2}}})

## Graph Walk aka Infeasibility Gliding (Exciting Stuff!)

**While the last two sections highlighted the elegant abstraction of grid generation and space transformation using `nimplex`, they did not really show the true power of the library or the novel combinatorics-based algorithm we developed in the associated paper. That is the simplex graph construction in arbitrary dimensions and the graph traversal it enables.**

**Let's start by generating the graph for our problem. We can use the `nimplex.embeddedpair_simplex_graph_fractional_py` function, which also first generates the two grids we already used in the previous sections.**

In [ ]:
_, _, graphN = nimplex.embeddedpair_simplex_graph_fractional_py(attainableSpaceComponentPositions, 12)

As you can see, we immediately obtain list of adjecent nodes (compostions) for each node in the graph, with few neighbors for the corners and many for the interior points. This is the power of the abstraction we are talking about!

In [ ]:
graphN[:5]

In [ ]:
graphN[200:205]

We can now use this graph to do a lot of things. Let's start by considering that **the infeasible region, for thermodynamic reasons beyond the scope of this tutorial, are generally difficult to predict but are generally bound by continous surfaces called phase boundaries.** 

**Thus, if we iteratively traverse the graph expanding only the feasible nodes, while noting infeasible nodes, we can glide along the phase boundaries and explore the feasible space without ever wasting our resources on calculations inside the insides of the infeasible regions.** 

Let's do this starting from the `W10.0 Nb10.0 Mo80.0` and terminating when we reach the `Hf95.0 Zr5.0` point. 

**Bonus Exercises:**
- Please note that you do not need to specify the termination point, as the algorithm will stop when it completeley explores the feasible space. In this example, we are just showing you how to specify it and it has no effect on the result. If you want to play a bit, you can set `endNode` to `12` to see how path to `W33.3 Nb33.3 Ta33.3` gets explored in just 87 nodes.
- If you remove the `endNode` termination (change `break` to `pass`), you can add the endNode (or any other node you believe is feasible) to the initial `queue` list and see how the algorithm explores the feasible space from multiple starting points much faster thanks to better parallelization.

In [ ]:
startingNode = 0
endNode = 90

print(f"Starting node: {formulas[startingNode]}")
print(f"Ending node: {formulas[endNode]}")

In [ ]:
gridFeasible = [None]*len(graphN)
queue = [0, 12, 90]
explored = set()
calcCount = 0

**Now, we will be using a `queue` of nodes to keep track of the nodes we need to visit and a set of visited nodes to avoid revisiting them. This simple procedure is a type of `depth-first search` algorithm that is guaranteed to find the shortest path between two points in a graph if it exists, while exploring the feasible space in the process in an unbiased way. In a more elaborate problem, you would likely want to implement a priority queue to explore the space in a more efficient way, but for this tutorial, this is more than enough and allows for better, more direct comparions with the typical complete exploration approach.** On the 4-core Codespaces VM you can expect it to take around 1-2 minutes. In a relatively simple problem like this one, the difference between the two approaches is not that big, coming partially from overhead of limited Python parallelization capabilities, but in a more complex problem, the difference can be dramatic.

In [ ]:
while len(queue)>0:
    print(f"Queue: {queue}")
    # Assign feasibilities to the current queue
    elPositions = [gridEl[i] for i in queue]
    if len(queue)>3:
        phases = process_map(equilibrium_callable, elPositions, max_workers=4)
    else:
        phases = [equilibrium_callable(elP) for elP in elPositions]
    feasibilities = [len(set(p) & set(infeasiblePhases))==0 and p!=[] for p in phases]

    calcCount += len(feasibilities)
    explored = explored.union(queue)

    # Create next queue based on neighbors of feasible points
    nextQueue = set()
    for f, i in zip(feasibilities, queue):
        gridFeasible[i] = f
        # Only if feasible
        if f:
            for n in graphN[i]:
                if n not in explored:
                    nextQueue.add(n)

    # Early termination criteria if we just evaluated the target
    if endNode in queue:
        pass

    print(f"Calculations done: {calcCount}")
    queue = list(nextQueue)

**You should now see that only 281 phase equilibria calculations were performed to get all the feasible points! That's only a bit more than half of what we had to do in the previous section. Let's plot the path we found in the 3D space.** If you rotate the figure, you can clearly see the path gliding along the infeasible space boundary.

In [ ]:
fig = px.scatter_3d(gridAtt_projected_df, x='x', y='y', z='z', color=gridFeasible, text=labels, hover_name=formulas,
              template='plotly_white', width=800, height=700, opacity=0.333, color_discrete_sequence=['green', 'red'],
              labels={'color':'Solid Solution Phases', 'x':'', 'y':'', 'z':''})
fig.update_scenes({'camera': {'eye': {'x': -2.3, 'y': 0.2, 'z': 0.2}}})

## Path Finding

**Now, let's take the appeal of graph representation a few steps forward! Since they are very general and widely applicable, there are countless off-the-shelf high-performance libraries implementing mathematically optimal path finding algorithms on graphs one can deploy on challenging problems.** 

In this example, for the sake of simplicity, we will use pure-Python `pathfinding` library which implements some of the most popular path finding algorithms without high performance (we do not need here) but also without any dependencies. In real applications, you would likely want to use a more advanced library like `networkx` which is more efficient and have more features, or write one yourself. Please note that `nimplex`'s CLI can generate massive graphs in native NumPy format you may find useful for such applications (e.g., to use Julia or C++ libraries).

In [ ]:
from pathfinding.core.graph import Graph
from pathfinding.finder.dijkstra import DijkstraFinder

Let's now turn our graph definition into a format that `pathfinding` library can understand, assigning equal cost to all edges. **Note, that we only need to consider the feasible points leading to other feasible point!**

In [ ]:
edges = []
for i, nList in enumerate(graphN):
    if gridFeasible[i]:
        for n in nList:
            if gridFeasible[n]:
                edges.append([i, n, 1])
print(edges[:5])

Let's initialize the `Graph` object.

In [ ]:
pathfindingGraph = Graph(edges=edges, bi_directional=False)

Now, let's create the `Finder` object that will perform the path finding. Here we use the `DijkstraFinder` which is a very popular algorithm for path finding in graphs (see [it's Wikipedia page](https://en.wikipedia.org/wiki/Dijkstra's_algorithm) for more details). It **finds the shortest possible path between two points in the graph**, which is exactly what we need here. In the future tutorials, we will use some more advanced algorithms, like A*, which accomplishes the same task but in the mathematcally minimal number of evaluations.

In [ ]:
finder = DijkstraFinder()

For hundreds of points, this shouldn't take more than tenth of a second or so.

In [ ]:
path, runs = finder.find_path(
    pathfindingGraph.node(0), 
    pathfindingGraph.node(90), 
    pathfindingGraph)

Let's now turn the path into a list of nodes on the path.

In [ ]:
shortestPath = [p.node_id for p in path]

And plot corresponding compositions. As you can see, we immediately get a neat list of compositions that we can use to design our additive manufacturing path.

In [ ]:
for step, i in enumerate(shortestPath):
    print(f"{step+1:>2}: {formulas[i]}")

**Even better, since we know the positions in the attainable design space, we can get a list of exact quantized instructions (e.g., powder flow rates) for the additive manufacturing apparatus to follow at each step.**

**This is a very powerful abstraction, eliminating possible mistakes and miscommunications between design and manufacturing teams. Furthermore, it also allows for easy manual (or mechanized) per-layer mixing of powders to create compositional paths in single-feed additive manufacturing systems.**

In [ ]:
integerGrid = nimplex.simplex_grid_py(4, 12)
for step, i in enumerate(shortestPath):
    print(f"{step+1:>2}: {integerGrid[i]}")

We can also plot the path in 3D like we did before! As you can see, it **glides through the inside of the tetrahedron in a shortest path** avoiding an obstacle extending far on the face.

In [ ]:
gridFeasibleMarked = ['path' if i in shortestPath else f for i, f in enumerate(gridFeasible)]
fig = px.scatter_3d(gridAtt_projected_df, x='x', y='y', z='z', color=gridFeasibleMarked, text=labels, hover_name=formulas,
              template='plotly_white', width=800, height=700, opacity=0.333, color_discrete_sequence=['blue', 'green', 'red'],
              labels={'color':'Solid Solution Phases', 'x':'', 'y':'', 'z':''})
fig.update_scenes({'camera': {'eye': {'x': -1.8, 'y': 1.2, 'z': 1.5}}})

## Path Planning considering the Property Gradient

**Let's go one step further to demonstrate the beaty of the graph representations in the context of efortless path planning in high-dimensional spaces by including the gradient into consideration in just a line or two of code. We will now stretch the space!**

In [ ]:
edges = []
gradientsList = []
penaltyFactor = 100
for i, nList in enumerate(graphN):
    if gridFeasible[i]:
        for n in nList:
            if gridFeasible[n]:
                rmsadGradient = abs(rmsadList[i]-rmsadList[n])
                gradientsList.append(rmsadGradient)
                edges.append([i, n, 1+round(rmsadGradient*penaltyFactor, 3)])
print(edges[:5])
print(f"{len(gradientsList)} gradients calculated with absolute value range between {min(gradientsList)} to {max(gradientsList)}")
print(f"Penalty at {penaltyFactor}x gradient *magnitude* per path step assigned.")

**As you can see, we just increased the predicted cost of moving between nodes by a small factor ranging from near 0 to 2.5 depending on the RMSAD gradient. In another words, we are prioritizing paths that change RMSAD the least over the entire path.** This can be done, e.g., to minimize the inter-layer stresses in the component caused by yield stress mismatch between the layers that may cause delamination. Similar considerations can also be made for other properties, like thermal expansion coefficient, missmatch of which causes internal stress or wrapping.

**Since the RMSAD is generally smoothly changing, we can expect that this will not impact the result in terms of number of nodes. Just finding a better path of equivalent length. However, if you try to square the gradient by changing penalty to `(rmsadGradient*penaltyFactor)**2`, you will see how viciously the path will avoid any RMSAD changes through zig-zag patterns.**

In [ ]:
pathfindingGraph = Graph(edges=edges, bi_directional=False)
finder = DijkstraFinder()
path, runs = finder.find_path(
    pathfindingGraph.node(0), 
    pathfindingGraph.node(90), 
    pathfindingGraph)
lowGradientPath = [p.node_id for p in path]

In [ ]:
for step, i in enumerate(lowGradientPath):
    print(f"{step+1:>2}: {formulas[i]}")

As one can see below, the path still is optimal (21 steps) but it now passes much closer to the center of the tetrahedron, avoiding the lower RMSAD region around `Mo33.3 Nb33.3 Ta33.3` from which it would have to climb a bit more rapidly to reach the `Hf95.0 Zr5.0` point.

In [ ]:
gridFeasibleMarked = ['path' if i in lowGradientPath else f for i, f in enumerate(gridFeasible)]
fig = px.scatter_3d(gridAtt_projected_df, x='x', y='y', z='z', color=gridFeasibleMarked, text=labels, hover_name=formulas,
              template='plotly_white', width=800, height=700, opacity=0.333, color_discrete_sequence=['blue', 'green', 'red'],
              labels={'color':'Solid Solution Phases', 'x':'', 'y':'', 'z':''})
fig.update_scenes({'camera': {'eye': {'x': -1.8, 'y': 1.2, 'z': 1.5}}})

## Property Maximization Along the Path

**Finally, let's demonstrate how easy it is to maximize a property along the path by exploiting the unidirectionaliy of the graph edge definitions in `nimplex`, which allow us to efortlessly enocde the property maximization goal as directional reward or penalty applied to edges.** This is as simple as applying the property gradient directly rather than its magnitude.

In [ ]:
edges = []
gradientsList = []
penaltyFactor = 20
for i, nList in enumerate(graphN):
    if gridFeasible[i]:
        for n in nList:
            if gridFeasible[n]:
                rmsadGradient = rmsadList[i]-rmsadList[n]
                gradientsList.append(rmsadGradient)
                edges.append([i, n, 1+round((rmsadGradient*penaltyFactor), 3)])
print(edges[:15])
print(f"{len(gradientsList)} gradients calculated with values up to {max(gradientsList)}")
print(f"Penalty at -{penaltyFactor}x gradient *value* per path step assigned (higher RMSAD preferred)")

In [ ]:
pathfindingGraph = Graph(edges=edges, bi_directional=False)
finder = DijkstraFinder()
path, runs = finder.find_path(
    pathfindingGraph.node(0), 
    pathfindingGraph.node(90), 
    pathfindingGraph)
pathList = [p.node_id for p in path]

In [ ]:
for step, i in enumerate(pathList):
    print(f"{step+1:>2}: {formulas[i]}")

**As you can see, we again complete the path in 21 steps but following a different path that maximizes the RMSAD along the way thanks to the small bias we introduced.**

In [ ]:
gridFeasibleMarked = ['path' if i in pathList else f for i, f in enumerate(gridFeasible)]
fig = px.scatter_3d(gridAtt_projected_df, x='x', y='y', z='z', color=gridFeasibleMarked, text=labels, hover_name=formulas,
              template='plotly_white', width=800, height=700, opacity=0.333, color_discrete_sequence=['blue', 'green', 'red'],
              labels={'color':'Solid Solution Phases', 'x':'', 'y':'', 'z':''})
fig.update_scenes({'camera': {'eye': {'x': -1.8, 'y': 1.2, 'z': 1.5}}})

**And, if we care more about going through high RMSAD regions than we do about the number of steps taken, we can always simply increase the bias to move the path into the high RMSAD regions even more at the cost of the number of steps taken.**

In [ ]:
edges = []
gradientsList = []
penaltyFactor = 200
for i, nList in enumerate(graphN):
    if gridFeasible[i]:
        for n in nList:
            if gridFeasible[n]:
                rmsadGradient = rmsadList[i]-rmsadList[n]
                gradientsList.append(rmsadGradient)
                edges.append([i, n, 1+round((rmsadGradient*penaltyFactor), 3)])
print(edges[:15])
print(f"{len(gradientsList)} gradients calculated with values up to {max(gradientsList)}")
print(f"Penalty at -{penaltyFactor}x gradient *value* per path step assigned (higher RMSAD preferred)")

pathfindingGraph = Graph(edges=edges, bi_directional=False)
finder = DijkstraFinder()
path, runs = finder.find_path(
    pathfindingGraph.node(0), 
    pathfindingGraph.node(90), 
    pathfindingGraph)
pathList = [p.node_id for p in path]

In [ ]:
for step, i in enumerate(pathList):
    print(f"{step+1:>2}: {formulas[i]}")

In [ ]:
gridFeasibleMarked = ['path' if i in pathList else f for i, f in enumerate(gridFeasible)]
fig = px.scatter_3d(gridAtt_projected_df, x='x', y='y', z='z', color=gridFeasibleMarked, text=labels, hover_name=formulas,
              template='plotly_white', width=800, height=700, opacity=0.333, color_discrete_sequence=['blue', 'green', 'red'],
              labels={'color':'Solid Solution Phases', 'x':'', 'y':'', 'z':''})
fig.update_scenes({'camera': {'eye': {'x': -1.8, 'y': 1.2, 'z': 1.5}}})

## Final Remarks

**And that's it! We hope you enjoyed this tutorial and that you see the potential of `nimplex` in your work. If you have any questions, feel free to ask them in the `nimplex` repository under the Issues tab. You are also welcome to shoot an email to the author at [adam@phaseslab.org](mailto:adam@phaseslab.org). We will be excited to hear about your applications.**

**We are planning on a couple more tutorials in the future, including a more advanced techniques for more specific use cases within materials science and beyond. PRs with your own tutorials are also welcome!**